# Market Basket Analysis

Since the data is so large and due to computational limit, we could not use MLxtend to do market basket analysis on the product level since there are a lot of products. But we can do MLxtend in department level or aisle level. There is another method to do market basket analysis on product level which is doing it manually (without MLxtend) as I have seen in https://pythondata.com/market-basket-analysis-with-python-and-pandas/. So in this notebook, I will use MLxtend to perform market basket analysis on aisles and I will perform market basket analysis on products without MLxtend. Then, a product can be recommended based on association rule.

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from mlxtend.frequent_patterns import apriori, fpmax, fpgrowth, association_rules
from mlxtend.preprocessing import TransactionEncoder

In [3]:
# read the data
df = pd.read_csv('df.csv')

In [7]:
df.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,aisle,department
0,2539329,1,prior,1,2,8,NaN,196.0,1.0,0.0,Soda,77.0,7.0,soft drinks,beverages
1,2539329,1,prior,1,2,8,NaN,14084.0,2.0,0.0,Organic Unsweetened Vanilla Almond Milk,91.0,16.0,soy lactosefree,dairy eggs
2,2539329,1,prior,1,2,8,NaN,12427.0,3.0,0.0,Original Beef Jerky,23.0,19.0,popcorn jerky,snacks
3,2539329,1,prior,1,2,8,NaN,26088.0,4.0,0.0,Aged White Cheddar Popcorn,23.0,19.0,popcorn jerky,snacks
4,2539329,1,prior,1,2,8,NaN,26405.0,5.0,0.0,XL Pick-A-Size Paper Towel Rolls,54.0,17.0,paper goods,household


## Apriori Method using MLxtend

In [4]:
# making all product into a list under the same order_id
basket_series = df.groupby('order_id')['aisle'].apply(list)

In [9]:
basket_series

order_id
1          [yogurt, other creams cheeses, fresh vegetable...
2          [eggs, fresh vegetables, spices seasonings, oi...
3          [yogurt, soy lactosefree, packaged vegetables ...
4          [breakfast bakery, cold flu allergy, energy gr...
5          [fresh fruits, salad dressing toppings, prepar...
                                 ...                        
3421079                                               [soap]
3421080    [milk, ice cream ice, juice nectars, fresh veg...
3421081    [chips pretzels, bread, condiments, packaged c...
3421082    [packaged poultry, butter, fresh fruits, packa...
3421083    [nuts seeds dried fruit, baby food formula, ba...
Name: aisle, Length: 3346083, dtype: object

In [6]:
# Fit the TransactionEncoder and transform the buckets
te = TransactionEncoder()
market = te.fit_transform(basket_series.values, sparse=True)
market_df = pd.DataFrame.sparse.from_spmatrix(market, columns=te.columns_)

In [7]:
# using max_len = 3: what is the 3 related items
x = apriori(market_df, min_support=0.04, use_colnames=True, max_len=3)
x["length"] = x.itemsets.apply(lambda x: len(x))
x

,support,itemsets,length
0,0.043115,(asian foods),1
1,0.045884,(baby food formula),1
2,0.076715,(baking ingredients),1
3,0.163958,(bread),1
4,0.068667,(breakfast bakery),1
...,...,...,...
221,0.047285,"(fresh vegetables, water seltzer sparkling wat...",3
222,0.087746,"(fresh vegetables, yogurt, packaged vegetables...",3
223,0.043522,"(milk, packaged cheese, packaged vegetables fr...",3
224,0.051020,"(milk, yogurt, packaged vegetables fruits)",3


In [9]:
rules_df = association_rules(x, metric="lift", min_threshold=1).sort_values('lift', ascending=False)

In [11]:
rules_df.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
491,(fresh herbs),"(fresh vegetables, packaged vegetables fruits)",0.093564,0.235156,0.047056,0.502929,2.138702,0.025054,1.538701
490,"(fresh vegetables, packaged vegetables fruits)",(fresh herbs),0.235156,0.093564,0.047056,0.200106,2.138702,0.025054,1.133195
341,(fresh herbs),"(fresh vegetables, fresh fruits)",0.093564,0.318137,0.061774,0.660230,2.075302,0.032008,2.006840
340,"(fresh vegetables, fresh fruits)",(fresh herbs),0.318137,0.093564,0.061774,0.194174,2.075302,0.032008,1.124853
489,"(fresh herbs, packaged vegetables fruits)",(fresh vegetables),0.052647,0.444341,0.047056,0.893802,2.011522,0.023663,5.232304


To read the table above(line 491): If a customer buys from fresh herbs department, then it is very likely that he/she buys fresh vegetables, packaged vegetables fruits. We can say that based on the 'lift' measurement. The higher the 'lift', the higher likelihood that 'consequents' will be bought after 'antecedents'.
<p>
We can see that fresh herbs, fresh vegetables, packaged vegetables fruits, and fresh fruits dominating the top 5. It means majority of customers bought products from those aisles. <p>

We can make some recommendations based on the item choose.

In [12]:
rules_df["antecedents"] = rules_df["antecedents"].apply(lambda x: list(x)).copy()
rules_df["consequents"] = rules_df["consequents"].apply(lambda x: list(x)).copy()

In [142]:
# Input basket
mybasket = ["milk", "bread"]

# metric
metric = "lift"

# make some recommendations based on the item on mybasket
def product_recs(basket, rule_df, metric):
    """ 
    Recommend product based on association rule
    Looking at which aisles the user bought, 
    we can give recommendations for the related aisles
    that the user might like,
    using lift as the metric
    """
    # Randomly select an item from the basket
    random_item = np.random.choice(basket, 1)[0]
    print(f"Since you like {random_item}, you might like: ")

    # Find rules where the item is in the antecedent
    rule_filter = rule_df["antecedents"].apply(lambda x: x[0]) == random_item

    # Filter the dataframe using rule_filter and sort by the selected metric
    filtered_df = rule_df[rule_filter].sort_values(by=metric, ascending=False)

    # Randomly return one of the top 20 items from the filtered dataframe
    reco = filtered_df.head(20).sample(10)["consequents"]

    return reco


product_recs(mybasket, rules_df, metric)


Since you like bread, you might like: 


229                  [fresh vegetables, fresh fruits]
5                                      [fresh fruits]
11                                  [packaged cheese]
243                                    [fresh fruits]
241                   [packaged cheese, fresh fruits]
258    [fresh vegetables, packaged vegetables fruits]
7                                  [fresh vegetables]
15                                           [yogurt]
9                                              [milk]
235                              [milk, fresh fruits]
Name: consequents, dtype: object

## Apriori Method Without MLxtend

We can do Market Basket Analysis from the product level by doing the calculations manually. I use the calculations from https://pythondata.com/market-basket-analysis-with-python-and-pandas/.

In [26]:
from itertools import combinations, groupby
from collections import Counter

In [27]:
# make a variable contained only order_id and product_name
order = df.set_index('order_id')['product_name']
order.head()

order_id
2539329                                       Soda
2539329    Organic Unsweetened Vanilla Almond Milk
2539329                        Original Beef Jerky
2539329                 Aged White Cheddar Popcorn
2539329           XL Pick-A-Size Paper Towel Rolls
Name: product_name, dtype: object

In [28]:
# calculate frequency
stats = order.value_counts().to_frame('frequency')

In [11]:
stats.head()

,frequency
Banana,491291
Bag of Organic Bananas,394930
Organic Strawberries,275577
Organic Baby Spinach,251705
Organic Hass Avocado,220877


In [29]:
# calc support
stats['support'] = stats/ len(set(order.index))*100
stats.head()

,frequency,support
Banana,491291,14.682571
Bag of Organic Bananas,394930,11.802756
Organic Strawberries,275577,8.235809
Organic Baby Spinach,251705,7.522378
Organic Hass Avocado,220877,6.601062


In [19]:
mean = np.mean(stats['support'])
mean

0.020342303020036267

In [20]:
max = np.max(stats['support'])
max

14.682570635575987

In [21]:
min = np.min(stats['support'])
min 

2.9885690223464273e-05

In [23]:
median = np.median(stats['support'])
median

0.001882798484078249

In [30]:
# min_support level
min_support = 0.018 #using the median
product_over_support = stats[stats['support'] >= min_support].index
orders_over_support = order[order.isin(product_over_support)]
orders_over_support

order_id
2539329                                       Soda
2539329    Organic Unsweetened Vanilla Almond Milk
2539329                        Original Beef Jerky
2539329                 Aged White Cheddar Popcorn
2539329           XL Pick-A-Size Paper Towel Rolls
                            ...                   
272231                      Fabric Softener Sheets
272231      Dark Chocolate Mint Snacking Chocolate
272231                    Phish Food Frozen Yogurt
272231                       French Baguette Bread
272231        Original Multigrain Spoonfuls Cereal
Name: product_name, Length: 29571452, dtype: object

In [31]:
#filter out order that only had 1 item order on the order_id
order_count = order.index.value_counts()
order_count.head()

1564244    145
790903     137
61355      127
2970392    121
2069920    116
Name: order_id, dtype: int64

In [32]:
order_morethan_2_id = order_count[order_count >=2 ].index
order_morethan_2_id

Int64Index([1564244,  790903,   61355, 2970392, 2069920, 3308010, 2753324,
            2499774,   77151, 2621625,
            ...
            1492978, 1057806, 2464327,  251361, 3066440, 2868157, 2468622,
            2039918, 3169669, 2226176],
           dtype='int64', length=3182490)

In [33]:
order_morethan_2 = order[order.index.isin(order_morethan_2_id)]
order_morethan_2

order_id
2539329                                       Soda
2539329    Organic Unsweetened Vanilla Almond Milk
2539329                        Original Beef Jerky
2539329                 Aged White Cheddar Popcorn
2539329           XL Pick-A-Size Paper Towel Rolls
                            ...                   
272231                      Fabric Softener Sheets
272231      Dark Chocolate Mint Snacking Chocolate
272231                    Phish Food Frozen Yogurt
272231                       French Baguette Bread
272231        Original Multigrain Spoonfuls Cereal
Name: product_name, Length: 33655513, dtype: object

In [34]:
#calc with the new dataframe
stats = order_morethan_2.value_counts().to_frame('frequency')
stats['support'] = stats / len(set(order_morethan_2.index)) *100
stats.head()

,frequency,support
Banana,489165,15.370512
Bag of Organic Bananas,392051,12.319002
Organic Strawberries,274416,8.622682
Organic Baby Spinach,250501,7.871227
Organic Hass Avocado,220158,6.917791


In [35]:
stats.shape

(49683, 2)

In [35]:
# create a func that will generate our itemsets and send our new order dataset through the generator
def generator(order):
    order = order.reset_index().values
    for order_id, product in groupby(order, lambda x: x[0]):
        product_list = [item[1] for item in product]
        for item_pair in combinations(product_list, 2):
            yield item_pair

itempair_gen = generator(order_morethan_2)
itempair_gen

<generator object generator at 0x16b38aa40>

In [36]:
itempair = pd.Series(Counter(itempair_gen)).to_frame('freqAC')
itempair['supportAC'] = itempair['freqAC'] / len(order_morethan_2_id) *100
itempair = itempair[itempair['supportAC'] >= min_support]
itempair.head()

freqAC  supportAC
Soda              Original Beef Jerky        576   0.018099
                  Bag of Organic Bananas    1560   0.049018
Honeycrisp Apples Bag of Organic Bananas     817   0.025672
Bartlett Pears    Bag of Organic Bananas    1317   0.041383
Soda              Zero Calorie Cola         2494   0.078366

In [37]:
# create table for association rules and compute relevant metrics
itempair = itempair.reset_index().rename(columns={'level_0' : 'antecedents', 'level_1': 'consequents'})
itempair = (itempair
     .merge(stats.rename(columns={'freq': 'freqA', 'support': 'antecedent support'}), left_on='antecedents', right_index=True)
     .merge(stats.rename(columns={'freq': 'freqC', 'support': 'consequents support'}), left_on='consequents', right_index=True))

In [38]:
itempair['confidenceAtoC'] = itempair['supportAC'] / itempair['antecedent support']
itempair['confidenceCtoA'] = itempair['supportAC'] / itempair['consequents support']
itempair['lift'] = itempair['supportAC'] / (itempair['antecedent support'] * itempair['consequents support'])
itempair = itempair[['antecedents', 'consequents','antecedent support', 'consequents support', 'confidenceAtoC','lift']]

In [94]:
itempair.head()

,antecedents,consequents,antecedent support,consequents support,confidenceAtoC,lift
0,Soda,Original Beef Jerky,1.085628,0.202891,0.016671,0.082170
18531,Bag of Organic Bananas,Original Beef Jerky,12.319002,0.202891,0.001724,0.008498
1,Soda,Bag of Organic Bananas,1.085628,12.319002,0.045152,0.003665
2,Honeycrisp Apples,Bag of Organic Bananas,0.449868,12.319002,0.057065,0.004632
3,Bartlett Pears,Bag of Organic Bananas,1.147781,12.319002,0.036055,0.002927


In [140]:
# to find items that have confidence > 0.005
conf = itempair
market_basket = conf[(conf.confidenceAtoC > 0.005)]
market_basket.set_index('antecedents', inplace = True)
market_basket.reset_index(inplace=True)
market_basket = market_basket.sort_values('lift', ascending = False)

In [41]:
market_basket

,antecedents,consequents,antecedent support,consequents support,confidenceAtoC,lift
16704,Oh My Yog! Pacific Coast Strawberry Trilayer Y...,Oh My Yog! Organic Wild Quebec Blueberry Cream...,0.091532,0.072962,0.300721,4.121625
16729,"Mighty 4 Kale, Strawberry, Amaranth & Greek Yo...","Mighty 4 Sweet Potato, Blueberry, Millet & Gre...",0.071265,0.082231,0.268959,3.270771
16712,Unsweetened Blackberry Water,Raspberry Essence Water,0.100644,0.065860,0.213862,3.247203
16728,"Mighty 4 Sweet Potato, Blueberry, Millet & Gre...","Mighty 4 Kale, Strawberry, Amaranth & Greek Yo...",0.082231,0.071265,0.221628,3.109913
16742,Cream Top Strawberry on the Bottom Yogurt,Cream Top Blueberry Yogurt,0.111799,0.054611,0.168072,3.077602
...,...,...,...,...,...,...
1128,Hass Avocados,Banana,1.619644,15.370512,0.030071,0.001956
156,Red Onion,Bag of Organic Bananas,1.394726,12.319002,0.023295,0.001891
5465,2% Reduced Fat Milk,Organic Strawberries,1.206445,8.622682,0.015705,0.001821
154,Green Onions,Bag of Organic Bananas,0.877646,12.319002,0.022341,0.001814


It seems that people who bought Oh My Yog! Pacific Coast Strawberry Trilayer Yogurt will buy Oh My Yog! Organic Wild Quebec Blueberry Cream Top Yogurt & Fruit. This is inferred from the high lift value of 4.12. It seems that customers like different flavors of the same products. The company could use the dataframe above to make some recommendations to customer or make some promotions to boost sale, such as 'Buy3 get 1 Free'.

In [141]:
# Input basket
mybasket = ['Banana', 'Raspberry Essence Water']

# metric
metric = "lift"

def product_recs(basket, rule_df, metric):
    """
    Recommend product based on association rule
    Args:
        basket: what item is in the basket 
        rule_df: market basket table
        metric: what metric you want to use
    returns: product recommendations
    """
    # Randomly select an item from the basket
    random_item = np.random.choice(basket, 1)[0]
    print(f"Since you like {random_item}, you might like: ")

    # Find rules where the item is in the antecedent
    rule_filter = rule_df["antecedents"].apply(lambda x: x) == random_item

    # Filter the dataframe using rule_filter and sort by the selected metric
    filtered_df = rule_df[rule_filter].sort_values(by=metric, ascending=False)

    # Randomly return one of the top 10 items from the filtered dataframe
    reco = filtered_df.head(10).sample(10)["consequents"]

    return reco


product_recs(mybasket, market_basket, metric)


Since you like Banana, you might like: 


15470                       Mandarin Oranges
13879                    Medium Navel Orange
1504                        Honeycrisp Apple
13203                             Black Plum
2673                      Organic Fuji Apple
15523       XL Emerald White Seedless Grapes
7746                          Bartlett Pears
1876                             Gala Apples
14351    Asparation/Broccolini/Baby Broccoli
12953           Large Yellow Flesh Nectarine
Name: consequents, dtype: object